In [1]:
%config Completer.use_jedi = False                                       
import warnings                                                             
warnings.filterwarnings(action="ignore")
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


***
다층 퍼셉트론
***

<img src="./다중퍼셉트론 (1).png">
<img src="./다중퍼셉트론 (2).png">
<img src="./다중퍼셉트론 (3).png">
<img src="./다중퍼셉트론 (4).png">

***
XOR 연산은 입력 데이터를 X라고 할 경우 [[0, 0], [0, 1], [1, 0], [1, 1]]과 같은 경우의 수가 있다.  
총 4개의 데이터가 각각 2개의 특성을 가지고 있으므로 X는 [4, 2]의 형태로 정의할 수 있다.  
입력값에 따른 출력값 Y는 [[0], [1], [1], [0]]이므로 Y는 [4, 1]의 형태로 정의할 수 있다.  
***

In [2]:
X = tf.placeholder( dtype=tf.float32, shape=(4, 2) ) #input; 학습데이터
Y = tf.placeholder( dtype=tf.float32, shape=(4, 1) ) #output; 레이블

***
첫번째 히든 레이어에서 2개의 입력(X)과 1개의 편향값(B1)을 받아서 2개의 시그모이드 출력(Z)을 2번째 히든 레이어로 전달한다.
***

In [3]:
# 2개의 입력을 받는 2개의 뉴런을 사용하기 위해서 가중치를 [건당 데이터 갯수, 뉴런의 갯수]만큼의 난수를 만든다.
W1 = tf.Variable( tf.random_normal([2, 2]) )

# 각 뉴런은 1개의 편향값(bias)를 갖는다.
# zeros() 메소드는 모든 원소의 값이 0인 텐서(2차원 이상의 행렬)를 생성한다.
B1 = tf.Variable( tf.zeros([2]) )

#시그모이드 함수를 이용해서 1번째 히든 레이어의 출력을 리턴한다.
Z = tf.sigmoid( tf.matmul(X, W1) + B1 )


In [4]:
sess = tf.Session()
sess.run( tf.global_variables_initializer() )
print( sess.run(W1) )
print( sess.run(B1) )
x_train =  [ [0, 0], [0, 1], [1, 0], [1, 1] ]
print( sess.run(Z, feed_dict={X : x_train}) )

[[ 1.3414143   0.27822655]
 [ 1.0373353  -0.40772948]]
[0. 0.]
[[0.5        0.5       ]
 [0.7383355  0.39945668]
 [0.79272246 0.56911135]
 [0.9151924  0.46766946]]


***
2번째 히든 레이어에서 1번째 히든 레이어의 출력값(Z)와 편향값(B2)을 받아서 1개의 시그모이드를 출력(Y_hat; 예측치)한다.; second_layer
***

In [5]:
# 2개의 Z(z1, z2)를 입력값으로 받는 1개의 뉴런을 만든다.
W2 = tf.Variable( tf.random_normal([2, 1]) )
B2 = tf.Variable( tf.zeros([1]) )
Y_hat = tf.sigmoid( tf.matmul(Z, W2) + B2 )

In [6]:
sess = tf.Session()
sess.run( tf.global_variables_initializer() )
print( sess.run(W1) )
print( sess.run(B1) )
x_train =  [ [0, 0], [0, 1], [1, 0], [1, 1] ]
print( sess.run(Y_hat, feed_dict={X : x_train}) )

[[-0.8999967   0.27934673]
 [-0.8819918  -0.42423123]]
[0. 0.]
[[0.45519188]
 [0.4492726 ]
 [0.4079763 ]
 [0.41144407]]


***
손실 함수로 크로스 엔트로피를 사용하고 경사 하강법으로 모델의 매개변수(가중치, 편향)를 최적화한다.
***

In [7]:
# 크로스 엔트로피
loss = tf.reduce_mean( -1 * (Y * tf.log(Y_hat) + (1.0 - Y) * tf.log(1 - Y_hat)) )

#경사 하강법
train_step = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

#모델 학습에 필요한 학습 데이터와 레이블을 만든다.
x_train =  [ [0, 0], [0, 1], [1, 0], [1, 1] ]
y_train = [ [0], [1], [1], [0] ]

***
학습시킨다. -> XOR연산은 값이 다르면 1을, 같으면 0
***

In [13]:
sess = tf.Session()
sess.run( tf.global_variables_initializer() )

print( '학습 데이터 -> {}'.format(x_train) )
print( '--------------------------------------' )

for epoch in range(10001):
    sess.run( train_step, feed_dict={X : x_train, Y : y_train} )
    if epoch % 2000 == 0:
        print( 'epoch -> {}'.format(epoch) )
        print( 'output\n', sess.run(Y_hat, feed_dict={X : x_train}), sep='' )
    #--------- if epoch
#------ for


학습 데이터 -> [[0, 0], [0, 1], [1, 0], [1, 1]]
--------------------------------------
epoch -> 0
output
[[0.3728474 ]
 [0.43581116]
 [0.3164098 ]
 [0.38311782]]
epoch -> 2000
output
[[0.53464913]
 [0.5050073 ]
 [0.5086148 ]
 [0.44488835]]
epoch -> 4000
output
[[0.3653035 ]
 [0.60865563]
 [0.6953234 ]
 [0.2713857 ]]
epoch -> 6000
output
[[0.1186188 ]
 [0.85205895]
 [0.89736056]
 [0.0947821 ]]
epoch -> 8000
output
[[0.05911748]
 [0.92383474]
 [0.9477594 ]
 [0.04831261]]
epoch -> 10000
output
[[0.03793159]
 [0.95048326]
 [0.96612906]
 [0.03132233]]
